In [1]:
# ============================================================
# WebP to MP4 - MINIMAL CLEAN VERSION (V10-MINI-UPDATED)
# ============================================================
from PIL import Image
import cv2
import numpy as np
import os
import argparse
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

VERBOSE = True  # Can be overridden via args

def print_banner(title, width=70):
    if VERBOSE:
        print("\n" + "=" * width)
        print(title.center(width))
        print("=" * width)

def print_progress(current, total, prefix="Processing"):
    if VERBOSE and (current % 10 == 0 or current + 1 == total):
        print(f" {prefix} {current + 1}/{total}")

# ============================================================
# ARGUMENT PARSING (SIMPLIFIED)
# ============================================================
def get_args(input_webp="ss.webp", **kwargs):
    global VERBOSE
    VERBOSE = kwargs.get('verbose', True)
    print_banner("CONFIGURATION / ARGUMENT SETUP")
    args = argparse.Namespace(
        input_webp=input_webp,
        output_mp4="ultimate_fixed_animation.mp4",
        sparkle_strength=kwargs.get('sparkle_strength', 0.15),  # Slightly lowered default
        static_threshold=kwargs.get('static_threshold', 10),
        semi_static_threshold=kwargs.get('semi_static_threshold', 35),
        median_window=kwargs.get('median_window', 3),
        transition_threshold=kwargs.get('transition_threshold', 2.5),
        blend_frames=kwargs.get('blend_frames', 5),
        crf=kwargs.get('crf', 18),
        enable_sparkle=kwargs.get('enable_sparkle', True),
        enable_gamma=kwargs.get('enable_gamma', True),
        fps_override=kwargs.get('fps_override', None),
        verbose=VERBOSE
    )
    if VERBOSE:
        print(f"Input WebP: {args.input_webp}")
        print(f"Output MP4: {args.output_mp4}")
        print(f"Sparkle strength: {args.sparkle_strength}")
        print(f"Static threshold: {args.static_threshold}")
        print(f"Semi-static threshold: {args.semi_static_threshold}")
        print(f"Median window: {args.median_window}")
        print(f"Transition threshold: {args.transition_threshold}")
        print(f"Blend frames: {args.blend_frames}")
        print(f"CRF quality: {args.crf}")
        print(f"Enable sparkle: {args.enable_sparkle}")
        print(f"Enable gamma: {args.enable_gamma}")
        print(f"FPS override: {args.fps_override or 'auto'}")
        print(f"Verbose mode: {args.verbose}")
    print("=" * 70 + "\n")
    return args

# ============================================================
# STEP 1: FRAME EXTRACTION
# ============================================================
def extract_frames(webp_path, output_dir="frames"):
    print_banner("STEP 1: EXTRACTING FRAMES FROM ANIMATED WEBP")
    print(f"Input file path: {webp_path}")
    print("Creating output directory...")
    os.makedirs(output_dir, exist_ok=True)
    im = Image.open(webp_path)
    durations = []
    print(f"Detected {im.n_frames} frames in WebP")
    print("Beginning extraction loop...")
    for i in range(im.n_frames):
        im.seek(i)
        frame_duration = im.info.get('duration', 66)
        durations.append(frame_duration)
        frame_path = f"{output_dir}/frame_{i:04d}.png"
        im.save(frame_path)
        print_progress(i, im.n_frames, "Extracted")
    avg_duration = np.median(durations)  # Use median for robustness
    print(f"\nExtraction complete!")
    print(f"Total frames extracted: {im.n_frames}")
    print(f"Median duration per frame: {avg_duration:.1f} ms")
    print(f"Estimated FPS: {1000 / avg_duration:.2f}")
    print_banner("FRAME EXTRACTION FINISHED", 50)
    return durations

# ============================================================
# IMPROVED SPARKLE REDUCTION (WITH VARIANCE SAFEGUARD)
# ============================================================
def detect_noise(frame):
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l = lab[:,:,0].astype(np.float32)
    high_freq = cv2.Laplacian(l, cv2.CV_32F)
    noise_var = np.var(high_freq)
    has_noise = noise_var > 50.0
    score = min(1.0, noise_var / 100.0)
    return {'has_noise': has_noise, 'score': score}

def reduce_jpeg_sparkles_enhanced(frames, base_strength=0.20):
    print_banner("STEP 2: ENHANCED JPEG SPARKLE / MOSQUITO NOISE REDUCTION")
    print(f"Using adaptive strength: {base_strength} (stronger edge protection, noise-adapted)")
    print(f"Analyzing {len(frames)} frames...")
    if len(frames) < 2:
        print("WARNING: Fewer than 2 frames - skipping")
        return frames
    # Adapt strength based on average noise score
    noise_scores = []
    sample_indices = np.linspace(0, len(frames)-1, min(10, len(frames)), dtype=int)
    for idx in sample_indices:
        noise_scores.append(detect_noise(frames[idx])['score'])
    avg_noise = np.mean(noise_scores)
    strength = min(0.40, base_strength * (1.0 + avg_noise * 0.5))  # Cap to prevent over-aggression
    print(f"Average noise score: {avg_noise:.2f} → Adjusted strength: {strength:.3f}")
    denoised = []
    for i, frame in enumerate(frames):
        start = max(0, i-1)
        end = min(len(frames), i+2)
        neighbors = frames[start:end]
        if len(neighbors) < 2:
            denoised.append(frame)
            continue
        stack = np.stack(neighbors, axis=0)
        temporal_median = np.median(stack, axis=0).astype(np.float32)
        frame_float = frame.astype(np.float32)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 30, 100)
        edge_mask = (edges > 0).astype(np.float32)
        edge_mask = cv2.GaussianBlur(edge_mask, (5, 5), 1.0)
        adaptive_strength = strength * (1.0 - edge_mask[:, :, np.newaxis] * 0.2)
        result = adaptive_strength * temporal_median + (1 - adaptive_strength) * frame_float
        result = np.clip(result, 0, 255)
        denoised.append(result.astype(np.uint8))
        print_progress(i, len(frames), "Denoised frame")
    original_var = np.std([np.mean(f) for f in frames])
    denoised_var = np.std([np.mean(f) for f in denoised])
    reduction = (1 - denoised_var / original_var) * 100 if original_var > 0 else 0
    if reduction < -5:
        print("WARNING: Denoising increased variance significantly → reverting to original frames")
        print_banner("SPARKLE REDUCTION SKIPPED (VARIANCE SAFEGUARD)", 50)
        return frames
    print(f"\nSparkle reduction complete! Variance reduced by {reduction:.1f}%")
    print_banner("SPARKLE REDUCTION FINISHED", 50)
    return denoised

# ============================================================
# GAMMA DETECTOR AND FIX (WITH IMPROVEMENT CHECK)
# ============================================================
def detect_gamma_mismatch(frame, ideal_gamma=2.2):
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l = lab[:, :, 0].flatten().astype(np.float32) / 255.0
    if len(l) == 0:
        return {'has_mismatch': False, 'score': 0.0}
    hist, _ = np.histogram(l, bins=64, range=(0,1), density=True)
    ideal_cum = np.linspace(0, 1, 65) ** ideal_gamma
    ideal_hist = np.diff(ideal_cum)
    bins = np.linspace(0, 1, 64)
    weights = np.exp(-6 * (bins - 0.5)**2)  # Less aggressive weighting
    total_dev = np.sum(np.abs(hist - ideal_hist) * weights)
    score = min(1.0, total_dev / 1.0)  # Higher denominator for tolerance
    has_mismatch = score > 0.75  # Higher threshold
    return {'has_mismatch': has_mismatch, 'score': score}

def improved_gamma_check(frames_before, frames_after, sample_count=8):
    if len(frames_before) != len(frames_after):
        return False, 0.0
    scores_before = []
    scores_after = []
    for i in np.linspace(0, len(frames_before)-1, min(sample_count, len(frames_before)), dtype=int):
        scores_before.append(detect_gamma_mismatch(frames_before[i])['score'])
        scores_after.append(detect_gamma_mismatch(frames_after[i])['score'])
    avg_before = np.mean(scores_before)
    avg_after = np.mean(scores_after)
    improvement = avg_before - avg_after
    print(f"Gamma score before: {avg_before:.3f}  after: {avg_after:.3f}  Δ: {improvement:+.3f}")
    return improvement > 0.08, improvement  # Threshold for "meaningful improvement"

def fix_gamma_mismatch(frames, profile=2):
    print_banner("GAMMA MISMATCH CORRECTION")
    print(f"Profile: {profile}")
    gamma = [None, 1.05, 1.10, 1.15, 1.20][profile]
    if gamma is None:
        print("Skipping")
        return frames
    fixed = []
    for i, frame in enumerate(frames):
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB).astype(np.float32)
        l = lab[:,:,0] / 255.0
        l_corrected = np.power(l, gamma) * 255.0
        lab[:,:,0] = np.clip(l_corrected, 0, 255)
        fixed.append(cv2.cvtColor(lab.astype(np.uint8), cv2.COLOR_LAB2BGR))
        print_progress(i, len(frames), "Fixed gamma")
    print(f"Gamma={gamma:.2f} on L only - applied")
    print_banner("GAMMA FIX FINISHED", 50)
    return fixed

# ============================================================
# GHOSTING DETECTOR AND FIX
# ============================================================
def detect_ghosting_artifacts(frame, prev_frame=None):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.float32)
    edges = cv2.Canny(gray.astype(np.uint8), 50, 150)
    edges_bool = edges.astype(bool)
    ghosting_score = 0
    for offset in [1, 2, 3]:
        shifted_h = np.roll(edges_bool, offset, axis=1)
        shifted_v = np.roll(edges_bool, offset, axis=0)
        h_overlap = np.sum(edges_bool & shifted_h)
        v_overlap = np.sum(edges_bool & shifted_v)
        total_edges = np.sum(edges_bool)
        if total_edges > 0:
            overlap_ratio = (h_overlap + v_overlap) / (total_edges * 2)
            if 0.3 < overlap_ratio < 0.7:
                ghosting_score += overlap_ratio
    return {'has_ghosting': ghosting_score > 1.0, 'score': ghosting_score}

def reduce_ghosting(frames):
    print_banner("TEMPORAL SMOOTHING FOR GHOSTING")
    return temporal_median_filter(frames, window=3)

# ============================================================
# METRICS
# ============================================================
def compute_metrics(frame_dir, name="frames"):
    frames = sorted([f for f in os.listdir(frame_dir) if f.endswith(".png")])
    frame_paths = [os.path.join(frame_dir, f) for f in frames]
    if not frame_paths:
        print(f"⚠️ No frames in {frame_dir}")
        return None, None, None
    arrays = []
    mean_luminances = []
    for path in frame_paths:
        arr = cv2.imread(path)
        if arr is None:
            continue
        arrays.append(arr)
        lab = cv2.cvtColor(arr, cv2.COLOR_BGR2LAB)
        l = lab[:, :, 0]
        mean_luminances.append(np.mean(l))
    brightness_std = np.std(mean_luminances)
    if len(arrays) > 1:
        stack = np.stack(arrays, axis=0)
        temporal_range = np.ptp(stack, axis=0)
        avg_flicker = np.mean(np.max(temporal_range, axis=-1))
        psnrs = []
        for i in range(1, len(arrays)):
            mse = np.mean((arrays[i-1].astype(np.float32) - arrays[i].astype(np.float32)) ** 2)
            if mse == 0:
                psnr = float('inf')
            else:
                psnr = 20 * np.log10(255.0 / np.sqrt(mse))
            psnrs.append(psnr)
        avg_psnr = np.mean(psnrs) if psnrs else 0.0
    else:
        avg_flicker = 0.0
        avg_psnr = 0.0
    print(f"📊 [{name}]")
    print(f" Brightness consistency (std): {brightness_std:.2f} (lower = better)")
    print(f" Temporal flicker (avg range): {avg_flicker:.2f} (lower = better)")
    print(f" Consecutive PSNR (dB): {avg_psnr:.2f} (higher = better)")
    return brightness_std, avg_flicker, avg_psnr

# ============================================================
# CORE FUNCTIONS
# ============================================================
def create_soft_mask(binary_mask, feather_radius=5):
    mask_uint8 = (binary_mask.astype(np.uint8) * 255)
    kernel_size = feather_radius * 2 + 1
    soft_mask = cv2.GaussianBlur(mask_uint8, (kernel_size, kernel_size), feather_radius / 2)
    return soft_mask.astype(np.float32) / 255.0

def temporal_median_filter(frames_bgr, window=3):
    if len(frames_bgr) < window:
        return frames_bgr
    filtered = []
    half_window = window // 2
    for i in range(len(frames_bgr)):
        start = max(0, i - half_window)
        end = min(len(frames_bgr), i + half_window + 1)
        window_frames = frames_bgr[start:end]
        if len(window_frames) > 0:
            lab_frames = [cv2.cvtColor(f, cv2.COLOR_BGR2LAB) for f in window_frames]
            stack_lab = np.stack(lab_frames, axis=0)
            median_lab = np.median(stack_lab, axis=0).astype(np.uint8)
            median_bgr = cv2.cvtColor(median_lab, cv2.COLOR_LAB2BGR)
            filtered.append(median_bgr)
        else:
            filtered.append(frames_bgr[i])
    return filtered

def detect_transition_flickers(frame_arrays, threshold=2.5):
    flicker_transitions = []
    brightness_values = []
    for f in frame_arrays:
        lab = cv2.cvtColor(f, cv2.COLOR_BGR2LAB)
        l_mean = np.mean(lab[:,:,0])
        brightness_values.append(l_mean)
    for i in range(1, len(frame_arrays)):
        prev_brightness = brightness_values[i-1]
        curr_brightness = brightness_values[i]
        jump = curr_brightness - prev_brightness
        if abs(jump) > threshold:
            context_start = max(0, i-3)
            context_end = min(len(frame_arrays), i+3)
            context_brightness = brightness_values[context_start:context_end]
            avg_context = np.mean(context_brightness)
            flicker_transitions.append({
                'frame_idx': i,
                'jump': jump,
                'prev_brightness': prev_brightness,
                'curr_brightness': curr_brightness,
                'context_brightness': avg_context
            })
    return flicker_transitions

def fix_transition_flickers_smart(frame_arrays, transitions, blend_frames=5):
    if not transitions:
        return
    print_banner("SMART BRIGHTNESS CORRECTION (ENHANCED, L-ONLY)")
    print(f" Found {len(transitions)} brightness jumps > 2.5")
    for trans in transitions:
        frame_idx = trans['frame_idx']
        jump = trans['jump']
        context_brightness = trans['context_brightness']
        curr_brightness = trans['curr_brightness']
        target_brightness = context_brightness
        correction_factor = (target_brightness - curr_brightness) / curr_brightness if curr_brightness > 0 else 0
        start_idx = max(0, frame_idx - 1)
        end_idx = min(len(frame_arrays), frame_idx + blend_frames)
        print(f" ✓ Fixing frame {frame_idx}: jump={jump:+.2f}, target={target_brightness:.1f}")
        for offset, idx in enumerate(range(start_idx, end_idx)):
            if idx >= len(frame_arrays):
                break
            distance_from_problem = abs(idx - frame_idx)
            blend_weight = max(0, 1.0 - (distance_from_problem / blend_frames))
            if blend_weight > 0:
                lab = cv2.cvtColor(frame_arrays[idx], cv2.COLOR_BGR2LAB).astype(np.float32)
                l = lab[:,:,0]
                l_corrected = l * (1.0 + correction_factor * blend_weight)
                original_l_mean = np.mean(l)
                corrected_l_mean = np.mean(l_corrected)
                if corrected_l_mean > 0:
                    contrast_preserved = l * (corrected_l_mean / original_l_mean)
                    l_final = 0.75 * contrast_preserved + 0.25 * l_corrected
                    lab[:,:,0] = np.clip(l_final, 0, 255)
                frame_arrays[idx] = cv2.cvtColor(lab.astype(np.uint8), cv2.COLOR_LAB2BGR)
        print(f" → Blended {end_idx - start_idx} frames with enhanced contrast preservation (L-only)")

def detect_motion_blur_in_frame(frame, prev_frame):
    if prev_frame is None:
        return False, 0.0
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    sharpness = laplacian.var()
    prev_laplacian = cv2.Laplacian(prev_gray, cv2.CV_64F)
    prev_sharpness = prev_laplacian.var()
    sharpness_loss = (prev_sharpness - sharpness) / prev_sharpness if prev_sharpness > 0 else 0
    has_blur = sharpness_loss > 0.25
    return has_blur, sharpness_loss

def sharpen_frame_adaptive(frame, amount=0.5):
    gaussian = cv2.GaussianBlur(frame, (0, 0), 2.0)
    sharpened = cv2.addWeighted(frame, 1.0 + amount, gaussian, -amount, 0)
    return sharpened

def process_frames_optimized(args, input_dir="frames", output_dir="processed", feather_radius=12):
    print_banner("PROCESSING FRAMES OPTIMIZED")
    os.makedirs(output_dir, exist_ok=True)
    frames = sorted([f for f in os.listdir(input_dir) if f.endswith(".png")])
    if not frames:
        print("❌ No frames in input_dir!")
        return
    frame_paths = [os.path.join(input_dir, f) for f in frames]
    print("\n=== STEP 1: Analyzing temporal characteristics ===")
    frame_arrays = []
    for path in frame_paths:
        arr = cv2.imread(path)
        if arr is not None:
            frame_arrays.append(arr)
    if len(frame_arrays) == 0:
        print("❌ No valid frames loaded!")
        return
    stack = np.stack(frame_arrays, axis=0)
    min_arr = np.min(stack, axis=0)
    max_arr = np.max(stack, axis=0)
    ptp = max_arr.astype(np.int32) - min_arr.astype(np.int32)
    max_ptp = np.max(ptp, axis=-1)
    static_mask = max_ptp <= args.static_threshold
    semi_static_mask = (max_ptp > args.static_threshold) & (max_ptp <= args.semi_static_threshold)
    motion_mask = max_ptp > args.semi_static_threshold
    static_mask_soft = create_soft_mask(static_mask, feather_radius)
    semi_static_mask_soft = create_soft_mask(semi_static_mask, feather_radius)
    static_pct = np.mean(static_mask) * 100
    semi_static_pct = np.mean(semi_static_mask) * 100
    motion_pct = np.mean(motion_mask) * 100
    print(f" Static pixels (stabilize): {static_pct:.1f}%")
    print(f" Semi-static (median filter): {semi_static_pct:.1f}%")
    print(f" Motion (preserve): {motion_pct:.1f}%")
    print(f" Mode: Motion-preserving stabilization")
    print("\n=== STEP 2: Computing stable reference ===")
    temporal_median = np.median(stack, axis=0).astype(np.float32)
    print(f" Using temporal median of all {len(frame_arrays)} frames")
    print(f"\n=== STEP 3: Applying temporal median filter (window={args.median_window}) ===")
    filtered_frames = temporal_median_filter(frame_arrays, window=args.median_window)
    print("\n=== STEP 4: Processing frames (motion-preserving) ===")
    processed_frames = []
    for idx, (original, filtered) in enumerate(zip(frame_arrays, filtered_frames)):
        original_float = original.astype(np.float32)
        filtered_float = filtered.astype(np.float32)
        frame_offset = np.zeros_like(original_float)
        for c in range(3):
            if np.any(static_mask):
                median_val = np.mean(temporal_median[:, :, c][static_mask])
                frame_val = np.mean(original_float[:, :, c][static_mask])
                offset = frame_val - median_val
                frame_offset[:, :, c] = offset
        result = original_float.copy()
        for c in range(3):
            stabilized = temporal_median[:, :, c] + frame_offset[:, :, c]
            result[:, :, c] = (
                static_mask_soft * stabilized +
                semi_static_mask_soft * filtered_float[:, :, c] +
                (1 - static_mask_soft - semi_static_mask_soft) * original_float[:, :, c]
            )
        result = np.clip(result, 0, 255).astype(np.uint8)
        processed_frames.append(result)
        print_progress(idx, len(frames), "Processed frame")
    transitions = detect_transition_flickers(processed_frames, threshold=args.transition_threshold)
    if transitions:
        fix_transition_flickers_smart(processed_frames, transitions, blend_frames=args.blend_frames)
    else:
        print(f"\n ✅ No transition flickers detected (all jumps < {args.transition_threshold})")
    print_banner("CHECKING FOR MOTION BLUR")
    blur_count = 0
    for idx in range(1, len(processed_frames)):
        has_blur, blur_amount = detect_motion_blur_in_frame(
            processed_frames[idx],
            processed_frames[idx-1]
        )
        if has_blur:
            processed_frames[idx] = sharpen_frame_adaptive(processed_frames[idx], amount=blur_amount * 0.5)
            blur_count += 1
    if blur_count > 0:
        print(f" ✓ Fixed {blur_count} frames with motion blur")
    else:
        print(f" ✅ No motion blur detected")
    for idx, result in enumerate(processed_frames):
        output_path = os.path.join(output_dir, frames[idx])
        cv2.imwrite(output_path, result)
    print(f"\n✅ All frames processed and saved")
    print_banner("FRAME PROCESSING FINISHED", 50)

# ============================================================
# REASSEMBLE MP4
# ============================================================
def reassemble_mp4(input_dir="processed", output_path="ultimate_fixed_animation.mp4",
                   durations=None, crf=18, fps_override=None):
    frames = sorted([os.path.join(input_dir, f) for f in os.listdir(input_dir)
                     if f.endswith(".png")])
    if not frames:
        print("❌ No frames found!")
        return False
    sample = cv2.imread(frames[0])
    if sample is None:
        print("❌ Cannot read first frame!")
        return False
    height, width, _ = sample.shape
    if durations and len(durations) > 0:
        avg_duration_ms = np.median(durations)  # Median for robustness
        fps = 1000 / avg_duration_ms if avg_duration_ms > 0 else 20
    else:
        fps = 20
    if fps_override:
        fps = fps_override
    print_banner("REASSEMBLING MP4")
    print(f" Resolution: {width}x{height}")
    print(f" FPS: {fps:.2f}")
    print(f" CRF: {crf} (quality)")
    print(f" Total frames: {len(frames)}")
    codecs_to_try = [
        ('mp4v', 'MPEG-4'),
        ('X264', 'H.264 (X264)'),
        ('H264', 'H.264 (H264)'),  # Added fallback
        ('avc1', 'H.264 (avc1)'),
    ]
    video = None
    for codec_code, codec_name in codecs_to_try:
        try:
            fourcc = cv2.VideoWriter_fourcc(*codec_code)
            video = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
            if video.isOpened():
                print(f" Codec: {codec_name} ✓")
                break
            else:
                video.release()
                video = None
        except:
            continue
    if video is None or not video.isOpened():
        print("❌ Failed to initialize video writer!")
        return False
    frames_written = 0
    for i, frame_path in enumerate(frames):
        img = cv2.imread(frame_path)
        if img is not None:
            video.write(img)
            frames_written += 1
        print_progress(i, len(frames), "Writing frame")
    video.release()
    if os.path.exists(output_path):
        file_size = os.path.getsize(output_path)
        if file_size > 0:
            print(f"✅ MP4 saved: {output_path}")
            print(f" File size: {file_size / 1024:.1f} KB")
            print(f" Frames written: {frames_written}/{len(frames)}")
            return True
        else:
            print(f"❌ MP4 file is 0 KB!")
            os.remove(output_path)
            return False
    return False

# ============================================================
# SIMPLIFIED QA (UPDATED: GAMMA VIA DELTA, GHOSTING, NOISE)
# ============================================================
def comprehensive_quality_check_enhanced(frames_dir, sample_count=10, gamma_before=None, gamma_after=None):
    print_banner("QUALITY ASSURANCE - SIMPLIFIED (GAMMA DELTA, GHOSTING, NOISE)")
    frames_list = sorted([f for f in os.listdir(frames_dir) if f.endswith('.png')])
    if len(frames_list) == 0:
        print("No frames!")
        return {}
    if len(frames_list) > sample_count:
        indices = np.linspace(0, len(frames_list)-1, sample_count, dtype=int)
        sampled = [frames_list[i] for i in indices]
    else:
        sampled = frames_list
    detections = {'ghosting': 0, 'noise': 0}
    prev_frame = None
    for frame_name in sampled:
        frame = cv2.imread(os.path.join(frames_dir, frame_name))
        if frame is None:
            continue
        ghosting_result = detect_ghosting_artifacts(frame, prev_frame)
        if ghosting_result['has_ghosting']:
            detections['ghosting'] += 1
        noise_result = detect_noise(frame)
        if noise_result['has_noise']:
            detections['noise'] += 1
        prev_frame = frame
        print_progress(sampled.index(frame_name), len(sampled), "QA")
    # Gamma handled via delta if provided
    if gamma_before is not None and gamma_after is not None:
        gamma_improved, delta = improved_gamma_check(gamma_before, gamma_after, sample_count)
        print(f" Gamma improvement: {'YES' if gamma_improved else 'NO'} (Δ: {delta:+.3f})")
    print(f"\nRESULTS:")
    for k, v in detections.items():
        pct = v / len(sampled) * 100 if len(sampled) > 0 else 0
        print(f" {k.replace('_', ' ').title()}: {v}/{len(sampled)} ({pct:.1f}%)")
    total = sum(detections.values())
    print(f"\nTotal issues: {total}")
    if total == 0:
        print("PERFECT")
    elif total <= 3:
        print("GOOD")
    else:
        print("REVIEW")
    print_banner("QA FINISHED", 50)
    return detections

# ============================================================
# MAIN PIPELINE
# ============================================================
if __name__ == "__main__":
    print_banner("WebP to MP4 - MINIMAL CLEAN VERSION (V10-MINI-UPDATED)")
    args = get_args("ss.webp")
    durations = extract_frames(args.input_webp)
    print("\n" + "="*70)
    print_banner("BEFORE PROCESSING QA (ORIGINAL FRAMES)")
    frame_files = sorted([f for f in os.listdir("frames") if f.endswith(".png")])
    frame_arrays = [cv2.imread(os.path.join("frames", f)) for f in frame_files if cv2.imread(os.path.join("frames", f)) is not None]
    detections_original = comprehensive_quality_check_enhanced("frames", sample_count=10)
    sample_len = min(10, len(frame_arrays))
    anti_ghosting = detections_original['ghosting'] / sample_len > 0.3 if sample_len > 0 else False
    print(f"Enabled fixes based on original QA (>30%):")
    print(f" - Anti-ghosting: {anti_ghosting}")
    print("\n" + "="*70)
    compute_metrics("frames", "ORIGINAL")
    print("="*70)
    desparkled = frame_arrays
    gamma_before = frame_arrays.copy()  # For delta check
    if args.enable_sparkle:
        desparkled = reduce_jpeg_sparkles_enhanced(frame_arrays, args.sparkle_strength)
    if args.enable_gamma:
        desparkled = fix_gamma_mismatch(desparkled, profile=1)
        print("Gamma fix applied after sparkle (profile=1)")
        # Run improvement check here
        _, _ = improved_gamma_check(gamma_before, desparkled, sample_count=10)
    if anti_ghosting:
        desparkled = reduce_ghosting(desparkled)
    for i, frame in enumerate(desparkled):
        cv2.imwrite(os.path.join("frames", frame_files[i]), frame)
    process_frames_optimized(args)
    print("\n" + "="*70)
    compute_metrics("processed", "PROCESSED")
    print("="*70)
    gamma_after = [cv2.imread(os.path.join("processed", f)) for f in sorted(os.listdir("processed")) if f.endswith(".png")]
    mp4_success = reassemble_mp4(input_dir="processed", output_path=args.output_mp4, durations=durations, crf=args.crf, fps_override=args.fps_override)
    if mp4_success:
        print_banner("AFTER PROCESSING QA (PROCESSED FRAMES)")
        comprehensive_quality_check_enhanced("processed", sample_count=10, gamma_before=gamma_before, gamma_after=gamma_after)
    print("\n" + "="*80)
    print("PIPELINE COMPLETE - MINIMAL CLEAN VERSION (UPDATED)")
    print("="*80)


        WebP to MP4 - MINIMAL CLEAN VERSION (V10-MINI-UPDATED)        

                    CONFIGURATION / ARGUMENT SETUP                    
Input WebP: ss.webp
Output MP4: ultimate_fixed_animation.mp4
Sparkle strength: 0.15
Static threshold: 10
Semi-static threshold: 35
Median window: 3
Transition threshold: 2.5
Blend frames: 5
CRF quality: 18
Enable sparkle: True
Enable gamma: True
FPS override: auto
Verbose mode: True


             STEP 1: EXTRACTING FRAMES FROM ANIMATED WEBP             
Input file path: ss.webp
Creating output directory...
Detected 48 frames in WebP
Beginning extraction loop...
 Extracted 1/48
 Extracted 11/48
 Extracted 21/48
 Extracted 31/48
 Extracted 41/48
 Extracted 48/48

Extraction complete!
Total frames extracted: 48
Median duration per frame: 40.0 ms
Estimated FPS: 25.00

            FRAME EXTRACTION FINISHED             


                BEFORE PROCESSING QA (ORIGINAL FRAMES)                

    QUALITY ASSURANCE - SIMPLIFIED (GAMMA DELTA, GHOSTING

In [8]:
# ============================================================
# WebP to MP4 - V13.1-PERFECTED (FINAL)
# ============================================================
from PIL import Image
import cv2
import numpy as np
import os
import argparse
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

VERBOSE = True

def print_banner(title, width=70):
    if VERBOSE:
        print("\n" + "=" * width)
        print(title.center(width))
        print("=" * width)

def print_progress(current, total, prefix="Processing"):
    if VERBOSE and (current % 10 == 0 or current + 1 == total):
        print(f" {prefix} {current + 1}/{total}")

# ============================================================
# DYNAMIC PARAMETER ANALYZER
# ============================================================
def analyze_frames_for_params(frame_arrays, sample_count=10):
    """
    Analyze frames and return optimal parameters based on content
    """
    print_banner("ANALYZING FRAMES FOR DYNAMIC PARAMETERS")
    
    sample_count = min(sample_count, len(frame_arrays))
    if sample_count == 0:
        print("No frames to analyze!")
        return None
    
    # Sample evenly across the video
    indices = np.linspace(0, len(frame_arrays)-1, sample_count, dtype=int)
    samples = [frame_arrays[i] for i in indices]
    
    metrics = {
        'edge_density': [],
        'color_variance': [],
        'noise_level': [],
        'temporal_variance': [],
        'saturation_mean': [],
        'brightness_range': [],
        'sharpness': []
    }
    
    print(f"Analyzing {sample_count} sample frames...")
    
    for i, frame in enumerate(samples):
        # Edge density
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 150)
        edge_density = np.mean(edges > 0)
        metrics['edge_density'].append(edge_density)
        
        # Color variance
        color_std = np.std(frame.reshape(-1, 3), axis=0).mean()
        metrics['color_variance'].append(color_std)
        
        # Noise level (Laplacian variance)
        laplacian = cv2.Laplacian(gray, cv2.CV_64F)
        noise_level = laplacian.var()
        metrics['noise_level'].append(noise_level)
        
        # Saturation
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        sat_mean = np.mean(hsv[:, :, 1]) / 255.0
        metrics['saturation_mean'].append(sat_mean)
        
        # Brightness range
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
        l_channel = lab[:, :, 0]
        brightness_range = np.ptp(l_channel)
        metrics['brightness_range'].append(brightness_range)
        
        # Sharpness
        sharpness = laplacian.var()
        metrics['sharpness'].append(sharpness)
        
        print_progress(i, sample_count, "Analyzed")
    
    # Calculate temporal variance (frame differences)
    if len(frame_arrays) > 1:
        temporal_diffs = []
        for i in range(1, min(len(frame_arrays), 20)):
            diff = np.mean(np.abs(frame_arrays[i].astype(np.float32) - 
                                  frame_arrays[i-1].astype(np.float32)))
            temporal_diffs.append(diff)
        metrics['temporal_variance'] = temporal_diffs
    
    # Average metrics
    avg_metrics = {
        'edge_density': np.mean(metrics['edge_density']),
        'color_variance': np.mean(metrics['color_variance']),
        'noise_level': np.mean(metrics['noise_level']),
        'temporal_variance': np.mean(metrics['temporal_variance']) if metrics['temporal_variance'] else 10.0,
        'saturation_mean': np.mean(metrics['saturation_mean']),
        'brightness_range': np.mean(metrics['brightness_range']),
        'sharpness': np.mean(metrics['sharpness'])
    }
    
    print(f"\n📊 Content Metrics:")
    print(f"  Edge Density: {avg_metrics['edge_density']:.4f}")
    print(f"  Color Variance: {avg_metrics['color_variance']:.2f}")
    print(f"  Noise Level: {avg_metrics['noise_level']:.2f}")
    print(f"  Temporal Variance: {avg_metrics['temporal_variance']:.2f}")
    print(f"  Saturation: {avg_metrics['saturation_mean']:.3f}")
    print(f"  Brightness Range: {avg_metrics['brightness_range']:.2f}")
    
    return avg_metrics

def calculate_dynamic_params(metrics, gamma_profile=1):
    """
    Calculate optimal parameters based on analyzed metrics
    """
    print_banner("CALCULATING DYNAMIC PARAMETERS")
    
    # Detect content type
    edge_density = metrics['edge_density']
    color_var = metrics['color_variance']
    saturation = metrics['saturation_mean']
    
    if edge_density > 0.15 and color_var < 25:
        content_type = "lineart"
    elif edge_density > 0.08 and saturation < 0.35:
        content_type = "anime"
    else:
        content_type = "photo"
    
    print(f"Content Type Detected: {content_type}")
    
    # Normalize noise level (0-1 scale)
    noise_normalized = min(1.0, metrics['noise_level'] / 1000.0)
    temporal_normalized = min(1.0, metrics['temporal_variance'] / 30.0)
    
    # A. Denoising Parameters
    base_strength = 0.12
    sparkle_strength = base_strength * (1.0 + noise_normalized * 0.5)
    sparkle_strength = np.clip(sparkle_strength, 0.10, 0.20)
    
    if content_type == "anime":
        max_denoise_strength = 0.25 + (noise_normalized * 0.15)
        max_denoise_strength = np.clip(max_denoise_strength, 0.25, 0.35)
    else:
        max_denoise_strength = 0.15 + (0.15 * noise_normalized)
        max_denoise_strength = np.clip(max_denoise_strength, 0.15, 0.30)
    
    # B. Edge Protection
    if content_type == "lineart":
        edge_protection = 0.50 + (edge_density * 1.0)
        edge_protection = np.clip(edge_protection, 0.50, 0.70)
    elif content_type == "anime":
        edge_protection = 0.35 + (edge_density * 0.7)
        edge_protection = np.clip(edge_protection, 0.35, 0.55)
    else:  # photo
        edge_protection = 0.20 + (edge_density * 0.6)
        edge_protection = np.clip(edge_protection, 0.20, 0.40)
    
    # C. Temporal thresholds
    static_threshold = 10
    semi_static_threshold = 35
    
    # D. Motion/Sharpening
    motion_blur_threshold = 0.25 + (0.10 * temporal_normalized)
    motion_blur_threshold = np.clip(motion_blur_threshold, 0.25, 0.35)
    
    sharpness_factor = min(1.0, metrics['sharpness'] / 500.0)
    max_sharpen_amount = 0.15 + (0.15 * (1.0 - sharpness_factor))
    max_sharpen_amount = np.clip(max_sharpen_amount, 0.15, 0.30)
    
    # E. Transition Handling
    transition_threshold = -1.0
    
    smoothness_factor = 1.0 - temporal_normalized
    blend_frames = 3 + int(4 * smoothness_factor)
    blend_frames = np.clip(blend_frames, 3, 7)
    
    # F. Median Window
    if temporal_normalized < 0.33:
        median_window = 3
    else:
        median_window = 5
    
    # G. Temporal Window for Sparkle Reduction
    if metrics['temporal_variance'] < 10.0:
        temporal_window = 5
    else:
        temporal_window = 3
    
    params = {
        'content_type': content_type,
        'sparkle_strength': float(sparkle_strength),
        'max_denoise_strength': float(max_denoise_strength),
        'edge_protection': float(edge_protection),
        'static_threshold': static_threshold,
        'semi_static_threshold': semi_static_threshold,
        'motion_blur_threshold': float(motion_blur_threshold),
        'max_sharpen_amount': float(max_sharpen_amount),
        'transition_threshold': transition_threshold,
        'blend_frames': int(blend_frames),
        'median_window': int(median_window),
        'temporal_window': int(temporal_window),
        'gamma_profile': gamma_profile,
        'preserve_edges': True,
        'auto_thresholds': True
    }
    
    print(f"\n🎯 Dynamic Parameters:")
    print(f"  Content Type: {params['content_type']}")
    print(f"  Sparkle Strength: {params['sparkle_strength']:.3f}")
    print(f"  Max Denoise Strength: {params['max_denoise_strength']:.3f}")
    print(f"  Edge Protection: {params['edge_protection']:.3f}")
    print(f"  Temporal Window: {params['temporal_window']} frames")
    print(f"  Motion Blur Threshold: {params['motion_blur_threshold']:.3f}")
    print(f"  Max Sharpen Amount: {params['max_sharpen_amount']:.3f}")
    print(f"  Blend Frames: {params['blend_frames']}")
    print(f"  Median Window: {params['median_window']}")
    print(f"  Gamma Profile: {params['gamma_profile']} (fixed)")
    
    print_banner("PARAMETER CALCULATION COMPLETE", 50)
    return params

# ============================================================
# ARGUMENT PARSING
# ============================================================
def get_args(input_webp="ss.webp", dynamic_mode=True, **kwargs):
    global VERBOSE
    VERBOSE = kwargs.get('verbose', True)
    print_banner("CONFIGURATION / ARGUMENT SETUP")
    
    args = argparse.Namespace(
        input_webp=input_webp,
        output_mp4=kwargs.get('output_mp4', "ultimate_fixed_animation.mp4"),
        
        dynamic_mode=dynamic_mode,
        
        sparkle_strength=kwargs.get('sparkle_strength', 0.15),
        max_denoise_strength=kwargs.get('max_denoise_strength', 0.25),
        static_threshold=kwargs.get('static_threshold', 10),
        semi_static_threshold=kwargs.get('semi_static_threshold', 35),
        auto_thresholds=kwargs.get('auto_thresholds', True),
        median_window=kwargs.get('median_window', 3),
        temporal_window=kwargs.get('temporal_window', 3),
        transition_threshold=kwargs.get('transition_threshold', 2.5),
        blend_frames=kwargs.get('blend_frames', 5),
        
        enable_sparkle=kwargs.get('enable_sparkle', True),
        enable_gamma=kwargs.get('enable_gamma', True),
        enable_bilateral_polish=kwargs.get('enable_bilateral_polish', True),
        gamma_profile=kwargs.get('gamma_profile', 1),
        
        motion_blur_threshold=kwargs.get('motion_blur_threshold', 0.30),
        max_sharpen_amount=kwargs.get('max_sharpen_amount', 0.30),
        
        content_type=kwargs.get('content_type', 'auto'),
        preserve_edges=kwargs.get('preserve_edges', True),
        edge_protection=kwargs.get('edge_protection', 0.5),
        
        crf=kwargs.get('crf', 18),
        fps_override=kwargs.get('fps_override', None),
        verbose=VERBOSE
    )
    
    if VERBOSE:
        print(f"Input WebP: {args.input_webp}")
        print(f"Output MP4: {args.output_mp4}")
        print(f"Dynamic Mode: {args.dynamic_mode}")
        print(f"Bilateral Polish: {args.enable_bilateral_polish}")
        print(f"Gamma profile: {args.gamma_profile} (fixed)")
        print(f"CRF quality: {args.crf}")
        print(f"Verbose mode: {args.verbose}")
    print("=" * 70 + "\n")
    return args

# ============================================================
# STEP 1: FRAME EXTRACTION
# ============================================================
def extract_frames(webp_path, output_dir="frames"):
    print_banner("STEP 1: EXTRACTING FRAMES FROM ANIMATED WEBP")
    print(f"Input file path: {webp_path}")
    print("Creating output directory...")
    os.makedirs(output_dir, exist_ok=True)
    im = Image.open(webp_path)
    durations = []
    print(f"Detected {im.n_frames} frames in WebP")
    print("Beginning extraction loop...")
    for i in range(im.n_frames):
        im.seek(i)
        frame_duration = im.info.get('duration', 66)
        durations.append(frame_duration)
        frame_path = f"{output_dir}/frame_{i:04d}.png"
        im.save(frame_path)
        print_progress(i, im.n_frames, "Extracted")
    avg_duration = np.median(durations)
    print(f"\nExtraction complete!")
    print(f"Total frames extracted: {im.n_frames}")
    print(f"Median duration per frame: {avg_duration:.1f} ms")
    print(f"Estimated FPS: {1000 / avg_duration:.2f}")
    print_banner("FRAME EXTRACTION FINISHED", 50)
    return durations

# ============================================================
# IMPROVED SPARKLE REDUCTION
# ============================================================
def adaptive_canny_thresholds(gray):
    v = np.median(gray)
    low = max(10, int(0.33 * v))
    high = min(255, int(0.66 * v))
    return low, high

def detect_noise(frame, content_type="photo"):
    """
    V13: Content-aware noise scoring (not just binary threshold)
    """
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l = lab[:,:,0].astype(np.float32)
    high_freq = cv2.Laplacian(l, cv2.CV_32F)
    noise_var = np.var(high_freq)
    
    # Content-aware scoring system
    if content_type == "anime":
        # Anime: 0-800 = clean, 800-1200 = moderate, 1200+ = noisy
        noise_score = max(0, min(1.0, (noise_var - 800) / 400))
        has_noise = noise_var > 1200.0
    elif content_type == "lineart":
        noise_score = max(0, min(1.0, (noise_var - 1000) / 500))
        has_noise = noise_var > 1500.0
    else:  # photo
        noise_score = max(0, min(1.0, noise_var / 200))
        has_noise = noise_var > 50.0
    
    return {
        'has_noise': has_noise,
        'score': noise_score,
        'variance': noise_var
    }

def reduce_jpeg_sparkles_enhanced(frames, base_strength=0.15,
                                  max_strength_cap=0.25,
                                  edge_protection_factor=0.5,
                                  content_type="photo",
                                  temporal_window=3):
    """
    V13: Relaxed variance safeguard for anime
    """
    print_banner("STEP 2: ENHANCED JPEG SPARKLE / MOSQUITO NOISE REDUCTION")
    print(f"Using base strength: {base_strength:.3f}")
    print(f"Max strength cap: {max_strength_cap:.3f}")
    print(f"Edge protection factor: {edge_protection_factor:.2f}")
    print(f"Temporal window: {temporal_window} frames")
    print(f"Content type: {content_type}")
    print(f"Analyzing {len(frames)} frames...")
    
    if len(frames) < 2:
        print("WARNING: Fewer than 2 frames - skipping")
        return frames
    
    noise_scores = []
    sample_indices = np.linspace(0, len(frames)-1, min(10, len(frames)), dtype=int)
    for idx in sample_indices:
        result = detect_noise(frames[idx], content_type)
        noise_scores.append(result['score'])
    
    avg_noise = np.mean(noise_scores)
    
    strength = base_strength * (1.0 + avg_noise * 0.5)
    strength = min(max_strength_cap, strength)
    print(f"Average noise score: {avg_noise:.2f} → Adjusted strength: {strength:.3f}")
    
    denoised = []
    half_window = temporal_window // 2
    
    for i, frame in enumerate(frames):
        start = max(0, i - half_window)
        end = min(len(frames), i + half_window + 1)
        neighbors = frames[start:end]
        
        if len(neighbors) < 2:
            denoised.append(frame)
            continue
        
        stack = np.stack(neighbors, axis=0)
        temporal_median = np.median(stack, axis=0).astype(np.float32)
        frame_float = frame.astype(np.float32)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        c_low, c_high = adaptive_canny_thresholds(gray)
        edges = cv2.Canny(gray, c_low, c_high)
        edge_mask = (edges > 0).astype(np.float32)
        edge_mask = cv2.GaussianBlur(edge_mask, (5, 5), 1.2)
        
        adaptive_strength = strength * (1.0 - edge_mask[:, :, np.newaxis] * edge_protection_factor)
        adaptive_strength = np.clip(adaptive_strength, 0.0, strength)
        
        result = adaptive_strength * temporal_median + (1 - adaptive_strength) * frame_float
        result = np.clip(result, 0, 255)
        denoised.append(result.astype(np.uint8))
        print_progress(i, len(frames), "Denoised frame")
    
    original_var = np.std([np.mean(f) for f in frames])
    denoised_var = np.std([np.mean(f) for f in denoised])
    reduction = (1 - denoised_var / original_var) * 100 if original_var > 0 else 0
    
    # V13: Relaxed variance safeguard for anime
    variance_threshold = -12.0 if content_type == "anime" else -5.0
    
    if reduction < variance_threshold:
        print(f"WARNING: Denoising variance change ({reduction:.1f}%) exceeds threshold ({variance_threshold}%) → reverting")
        print_banner("SPARKLE REDUCTION SKIPPED (VARIANCE SAFEGUARD)", 50)
        return frames
    
    print(f"\nSparkle reduction complete! Variance change: {reduction:.1f}%")
    print(f"Variance threshold: {variance_threshold}% ({'anime-optimized' if content_type == 'anime' else 'photo'})")
    print_banner("SPARKLE REDUCTION FINISHED", 50)
    return denoised

# ============================================================
# GAMMA FIXER ONLY
# ============================================================
def fix_gamma_mismatch(frames, profile=1):
    print_banner("GAMMA MISMATCH CORRECTION")
    print(f"Profile: {profile}")
    gamma_table = {1: 1.05, 2: 1.10}
    gamma = gamma_table.get(profile, 1.05)
    print(f"Using gamma={gamma:.2f}")
    
    fixed = []
    for i, frame in enumerate(frames):
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB).astype(np.float32)
        l = lab[:,:,0] / 255.0
        l_corrected = np.power(l, gamma) * 255.0
        lab[:,:,0] = np.clip(l_corrected, 0, 255)
        fixed.append(cv2.cvtColor(lab.astype(np.uint8), cv2.COLOR_LAB2BGR))
        print_progress(i, len(frames), "Fixed gamma")
    
    print(f"Gamma={gamma:.2f} on L only - applied")
    print_banner("GAMMA FIX FINISHED", 50)
    return fixed

# ============================================================
# V13.1: BILATERAL POLISH PASS (PERFECTED)
# ============================================================
def apply_bilateral_polish(frames, static_mask, semi_static_mask, content_type="photo"):
    """
    V13.1: Apply very light bilateral filter only on flat areas
    IMPROVED: Dynamic sigma based on content type + float32 safety
    """
    print_banner("BILATERAL POLISH PASS (FLAT AREAS ONLY)")
    print(f"Processing {len(frames)} frames with selective bilateral filtering...")
    
    # V13.1 IMPROVEMENT #1: Dynamic sigma based on content type
    sigma = 35 if content_type == "anime" else 50
    print(f"Bilateral sigma: {sigma} ({'anime-optimized' if content_type == 'anime' else 'photo'})")
    
    # Combine masks: full strength on static, half on semi-static
    combined_mask = np.clip(static_mask + semi_static_mask * 0.5, 0, 1)
    mask_3d = combined_mask[:, :, np.newaxis]
    
    static_pct = np.mean(static_mask) * 100
    semi_static_pct = np.mean(semi_static_mask) * 100
    total_pct = np.mean(combined_mask > 0.1) * 100
    
    print(f"Bilateral will affect:")
    print(f"  Static areas: {static_pct:.1f}% (full strength)")
    print(f"  Semi-static areas: {semi_static_pct:.1f}% (half strength)")
    print(f"  Total coverage: {total_pct:.1f}%")
    print(f"  Motion areas: {100-total_pct:.1f}% (untouched)")
    
    polished = []
    for i, frame in enumerate(frames):
        # Apply bilateral filter with dynamic sigma
        bilateral = cv2.bilateralFilter(frame, d=5, sigmaColor=sigma, sigmaSpace=sigma)
        
        # V13.1 IMPROVEMENT #2: Explicit float32 for safe blending
        bilateral_float = bilateral.astype(np.float32)
        frame_float = frame.astype(np.float32)
        
        # Blend only where mask is active
        result = (mask_3d * bilateral_float + (1 - mask_3d) * frame_float)
        result = np.clip(result, 0, 255).astype(np.uint8)
        polished.append(result)
        
        print_progress(i, len(frames), "Polished frame")
    
    print(f"\n✅ Bilateral polish complete!")
    print_banner("BILATERAL POLISH FINISHED", 50)
    return polished

# ============================================================
# GHOSTING DETECTOR AND FIX
# ============================================================
def detect_ghosting_artifacts(frame, prev_frame=None):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.float32)
    edges = cv2.Canny(gray.astype(np.uint8), 50, 150)
    edges_bool = edges.astype(bool)
    ghosting_score = 0
    for offset in [1, 2, 3]:
        shifted_h = np.roll(edges_bool, offset, axis=1)
        shifted_v = np.roll(edges_bool, offset, axis=0)
        h_overlap = np.sum(edges_bool & shifted_h)
        v_overlap = np.sum(edges_bool & shifted_v)
        total_edges = np.sum(edges_bool)
        if total_edges > 0:
            overlap_ratio = (h_overlap + v_overlap) / (total_edges * 2)
            if 0.3 < overlap_ratio < 0.7:
                ghosting_score += overlap_ratio
    return {'has_ghosting': ghosting_score > 1.0, 'score': ghosting_score}

def reduce_ghosting(frames):
    print_banner("TEMPORAL SMOOTHING FOR GHOSTING")
    return temporal_median_filter(frames, window=3)

# ============================================================
# METRICS
# ============================================================
def compute_metrics(frame_dir, name="frames"):
    frames = sorted([f for f in os.listdir(frame_dir) if f.endswith(".png")])
    frame_paths = [os.path.join(frame_dir, f) for f in frames]
    if not frame_paths:
        print(f"⚠️ No frames in {frame_dir}")
        return None, None, None
    
    arrays = []
    mean_luminances = []
    for path in frame_paths:
        arr = cv2.imread(path)
        if arr is None:
            continue
        arrays.append(arr)
        lab = cv2.cvtColor(arr, cv2.COLOR_BGR2LAB)
        l = lab[:, :, 0]
        mean_luminances.append(np.mean(l))
    
    brightness_std = np.std(mean_luminances)
    
    if len(arrays) > 1:
        stack = np.stack(arrays, axis=0)
        temporal_range = np.ptp(stack, axis=0)
        avg_flicker = np.mean(np.max(temporal_range, axis=-1))
        
        psnrs = []
        for i in range(1, len(arrays)):
            mse = np.mean((arrays[i-1].astype(np.float32) - arrays[i].astype(np.float32)) ** 2)
            if mse == 0:
                psnr = float('inf')
            else:
                psnr = 20 * np.log10(255.0 / np.sqrt(mse))
            psnrs.append(psnr)
        avg_psnr = np.mean(psnrs) if psnrs else 0.0
    else:
        avg_flicker = 0.0
        avg_psnr = 0.0
    
    print(f"📊 [{name}]")
    print(f" Brightness consistency (std): {brightness_std:.2f} (lower = better)")
    print(f" Temporal flicker (avg range): {avg_flicker:.2f} (lower = better)")
    print(f" Consecutive PSNR (dB): {avg_psnr:.2f} (higher = better)")
    return brightness_std, avg_flicker, avg_psnr

# ============================================================
# CORE FUNCTIONS
# ============================================================
def create_soft_mask(binary_mask, feather_radius=5):
    mask_uint8 = (binary_mask.astype(np.uint8) * 255)
    kernel_size = feather_radius * 2 + 1
    soft_mask = cv2.GaussianBlur(mask_uint8, (kernel_size, kernel_size), feather_radius / 2)
    return soft_mask.astype(np.float32) / 255.0

def temporal_median_filter(frames_bgr, window=3):
    if len(frames_bgr) < window:
        return frames_bgr
    filtered = []
    half_window = window // 2
    for i in range(len(frames_bgr)):
        start = max(0, i - half_window)
        end = min(len(frames_bgr), i + half_window + 1)
        window_frames = frames_bgr[start:end]
        if len(window_frames) > 0:
            lab_frames = [cv2.cvtColor(f, cv2.COLOR_BGR2LAB) for f in window_frames]
            stack_lab = np.stack(lab_frames, axis=0)
            median_lab = np.median(stack_lab, axis=0).astype(np.uint8)
            median_bgr = cv2.cvtColor(median_lab, cv2.COLOR_LAB2BGR)
            filtered.append(median_bgr)
        else:
            filtered.append(frames_bgr[i])
    return filtered

def detect_transition_flickers(frame_arrays, threshold=2.5):
    flicker_transitions = []
    brightness_values = []
    for f in frame_arrays:
        lab = cv2.cvtColor(f, cv2.COLOR_BGR2LAB)
        l_mean = np.mean(lab[:,:,0])
        brightness_values.append(l_mean)
    
    if threshold < 0:
        std_b = np.std(brightness_values)
        threshold = max(1.5, 2.0 * std_b)
        print(f" Auto transition threshold: {threshold:.2f}")
    
    for i in range(1, len(frame_arrays)):
        prev_brightness = brightness_values[i-1]
        curr_brightness = brightness_values[i]
        jump = curr_brightness - prev_brightness
        if abs(jump) > threshold:
            context_start = max(0, i-3)
            context_end = min(len(frame_arrays), i+3)
            context_brightness = brightness_values[context_start:context_end]
            avg_context = np.mean(context_brightness)
            flicker_transitions.append({
                'frame_idx': i,
                'jump': jump,
                'prev_brightness': prev_brightness,
                'curr_brightness': curr_brightness,
                'context_brightness': avg_context
            })
    return flicker_transitions

def fix_transition_flickers_smart(frame_arrays, transitions, blend_frames=5):
    if not transitions:
        return
    print_banner("SMART BRIGHTNESS CORRECTION (ENHANCED, L-ONLY)")
    print(f" Found {len(transitions)} brightness jumps")
    blend_frames = max(3, min(blend_frames, 7))
    
    for trans in transitions:
        frame_idx = trans['frame_idx']
        jump = trans['jump']
        context_brightness = trans['context_brightness']
        curr_brightness = trans['curr_brightness']
        target_brightness = context_brightness
        correction_factor = (target_brightness - curr_brightness) / curr_brightness if curr_brightness > 0 else 0
        correction_factor = np.clip(correction_factor, -0.2, 0.2)
        
        start_idx = max(0, frame_idx - 1)
        end_idx = min(len(frame_arrays), frame_idx + blend_frames)
        print(f" ✓ Fixing frame {frame_idx}: jump={jump:+.2f}, corr={correction_factor:+.3f}")
        
        for idx in range(start_idx, end_idx):
            if idx >= len(frame_arrays):
                break
            distance_from_problem = abs(idx - frame_idx)
            blend_weight = max(0, 1.0 - (distance_from_problem / blend_frames))
            if blend_weight > 0:
                lab = cv2.cvtColor(frame_arrays[idx], cv2.COLOR_BGR2LAB).astype(np.float32)
                l = lab[:,:,0]
                l_corrected = l * (1.0 + correction_factor * blend_weight)
                original_l_mean = np.mean(l)
                corrected_l_mean = np.mean(l_corrected)
                if corrected_l_mean > 0 and original_l_mean > 0:
                    contrast_preserved = l * (corrected_l_mean / original_l_mean)
                    l_final = 0.75 * contrast_preserved + 0.25 * l_corrected
                    lab[:,:,0] = np.clip(l_final, 0, 255)
                frame_arrays[idx] = cv2.cvtColor(lab.astype(np.uint8), cv2.COLOR_LAB2BGR)

def detect_motion_blur_in_frame(frame, prev_frame):
    if prev_frame is None:
        return False, 0.0
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    sharpness = laplacian.var()
    prev_laplacian = cv2.Laplacian(prev_gray, cv2.CV_64F)
    prev_sharpness = prev_laplacian.var()
    sharpness_loss = (prev_sharpness - sharpness) / prev_sharpness if prev_sharpness > 0 else 0
    has_blur = sharpness_loss > 0.25
    return has_blur, sharpness_loss

def sharpen_frame_adaptive(frame, amount=0.3):
    amount = max(0.0, min(amount, 0.3))
    gaussian = cv2.GaussianBlur(frame, (0, 0), 2.0)
    sharpened = cv2.addWeighted(frame, 1.0 + amount, gaussian, -amount, 0)
    return sharpened

def process_frames_optimized(args, input_dir="frames", output_dir="processed", feather_radius=12):
    """
    V13: Returns masks for bilateral polish
    """
    print_banner("PROCESSING FRAMES OPTIMIZED")
    os.makedirs(output_dir, exist_ok=True)
    frames = sorted([f for f in os.listdir(input_dir) if f.endswith(".png")])
    if not frames:
        print("❌ No frames in input_dir!")
        return None, None
    
    frame_paths = [os.path.join(input_dir, f) for f in frames]
    print("\n=== STEP 1: Analyzing temporal characteristics ===")
    frame_arrays = []
    for path in frame_paths:
        arr = cv2.imread(path)
        if arr is not None:
            frame_arrays.append(arr)
    
    if len(frame_arrays) == 0:
        print("❌ No valid frames loaded!")
        return None, None
    
    stack = np.stack(frame_arrays, axis=0)
    min_arr = np.min(stack, axis=0)
    max_arr = np.max(stack, axis=0)
    ptp = max_arr.astype(np.int32) - min_arr.astype(np.int32)
    max_ptp = np.max(ptp, axis=-1)
    
    static_th = args.static_threshold
    semi_static_th = args.semi_static_threshold
    
    if args.auto_thresholds:
        vals = max_ptp.flatten()
        static_th = np.percentile(vals, 10)
        semi_static_th = np.percentile(vals, 50)
        static_th = float(np.clip(static_th, 5, 50))
        semi_static_th = float(np.clip(semi_static_th, 20, 80))
        print(f" Auto static_threshold: {static_th:.2f}")
        print(f" Auto semi_static_threshold: {semi_static_th:.2f}")
    
    static_mask = max_ptp <= static_th
    semi_static_mask = (max_ptp > static_th) & (max_ptp <= semi_static_th)
    motion_mask = max_ptp > semi_static_th
    
    static_mask_soft = create_soft_mask(static_mask, feather_radius)
    semi_static_mask_soft = create_soft_mask(semi_static_mask, feather_radius)
    
    static_pct = np.mean(static_mask) * 100
    semi_static_pct = np.mean(semi_static_mask) * 100
    motion_pct = np.mean(motion_mask) * 100
    print(f" Static pixels (stabilize): {static_pct:.1f}%")
    print(f" Semi-static (median filter): {semi_static_pct:.1f}%")
    print(f" Motion (preserve): {motion_pct:.1f}%")
    
    print("\n=== STEP 2: Computing stable reference ===")
    temporal_median = np.median(stack, axis=0).astype(np.float32)
    print(f" Using temporal median of all {len(frame_arrays)} frames")
    
    print(f"\n=== STEP 3: Applying temporal median filter (window={args.median_window}) ===")
    filtered_frames = temporal_median_filter(frame_arrays, window=args.median_window)
    
    print("\n=== STEP 4: Processing frames (motion-preserving) ===")
    processed_frames = []
    for idx, (original, filtered) in enumerate(zip(frame_arrays, filtered_frames)):
        original_float = original.astype(np.float32)
        filtered_float = filtered.astype(np.float32)
        frame_offset = np.zeros_like(original_float)
        
        for c in range(3):
            if np.any(static_mask):
                median_val = np.mean(temporal_median[:, :, c][static_mask])
                frame_val = np.mean(original_float[:, :, c][static_mask])
                offset = frame_val - median_val
                frame_offset[:, :, c] = offset
        
        result = original_float.copy()
        for c in range(3):
            stabilized = temporal_median[:, :, c] + frame_offset[:, :, c]
            result[:, :, c] = (
                static_mask_soft * stabilized +
                semi_static_mask_soft * filtered_float[:, :, c] +
                (1 - static_mask_soft - semi_static_mask_soft) * original_float[:, :, c]
            )
        
        result = np.clip(result, 0, 255).astype(np.uint8)
        processed_frames.append(result)
        print_progress(idx, len(frames), "Processed frame")
    
    trans_threshold = args.transition_threshold
    if args.transition_threshold <= 0:
        trans_threshold = -1.0
    
    transitions = detect_transition_flickers(processed_frames, threshold=trans_threshold)
    if transitions:
        fix_transition_flickers_smart(processed_frames, transitions, blend_frames=args.blend_frames)
    else:
        print(f"\n ✅ No transition flickers detected")
    
    print_banner("CHECKING FOR MOTION BLUR")
    blur_count = 0
    for idx in range(1, len(processed_frames)):
        has_blur, blur_amount = detect_motion_blur_in_frame(
            processed_frames[idx],
            processed_frames[idx-1]
        )
        if has_blur and blur_amount > args.motion_blur_threshold:
            sharpen_amount = min(args.max_sharpen_amount, blur_amount * 0.5)
            processed_frames[idx] = sharpen_frame_adaptive(processed_frames[idx], amount=sharpen_amount)
            blur_count += 1
    
    if blur_count > 0:
        print(f" ✓ Fixed {blur_count} frames with motion blur")
    else:
        print(f" ✅ No motion blur detected")
    
    for idx, result in enumerate(processed_frames):
        output_path = os.path.join(output_dir, frames[idx])
        cv2.imwrite(output_path, result)
    
    print(f"\n✅ All frames processed and saved")
    print_banner("FRAME PROCESSING FINISHED", 50)
    
    return static_mask_soft, semi_static_mask_soft

# ============================================================
# V13.1: REASSEMBLE MP4 (IMPROVED CODEC ORDER)
# ============================================================
def reassemble_mp4(input_dir="processed", output_path="ultimate_fixed_animation.mp4",
                   durations=None, crf=18, fps_override=None):
    frames = sorted([os.path.join(input_dir, f) for f in os.listdir(input_dir)
                     if f.endswith(".png")])
    if not frames:
        print("❌ No frames found!")
        return False
    
    sample = cv2.imread(frames[0])
    if sample is None:
        print("❌ Cannot read first frame!")
        return False
    
    height, width, _ = sample.shape
    if durations and len(durations) > 0:
        avg_duration_ms = np.median(durations)
        fps = 1000 / avg_duration_ms if avg_duration_ms > 0 else 20
    else:
        fps = 20
    
    if fps_override:
        fps = fps_override
    
    print_banner("REASSEMBLING MP4")
    print(f" Resolution: {width}x{height}")
    print(f" FPS: {fps:.2f}")
    print(f" CRF: {crf} (quality)")
    print(f" Total frames: {len(frames)}")
    
    # V13.1 IMPROVEMENT #3: Better codec order (H.264 first)
    codecs_to_try = [
        ('avc1', 'H.264 (avc1)'),      # Best: modern H.264, better compression
        ('X264', 'H.264 (X264)'),      # Good: cross-platform H.264
        ('H264', 'H.264 (H264)'),      # Backup H.264
        ('mp4v', 'MPEG-4'),            # Fallback: older but compatible
    ]
    
    video = None
    for codec_code, codec_name in codecs_to_try:
        try:
            fourcc = cv2.VideoWriter_fourcc(*codec_code)
            video = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
            if video.isOpened():
                print(f" Codec: {codec_name} ✓")
                break
            else:
                video.release()
                video = None
        except:
            continue
    
    if video is None or not video.isOpened():
        print("❌ Failed to initialize video writer!")
        return False
    
    frames_written = 0
    for i, frame_path in enumerate(frames):
        img = cv2.imread(frame_path)
        if img is not None:
            video.write(img)
            frames_written += 1
        print_progress(i, len(frames), "Writing frame")
    
    video.release()
    
    if os.path.exists(output_path):
        file_size = os.path.getsize(output_path)
        if file_size > 0:
            print(f"✅ MP4 saved: {output_path}")
            print(f" File size: {file_size / 1024:.1f} KB")
            print(f" Frames written: {frames_written}/{len(frames)}")
            return True
        else:
            print(f"❌ MP4 file is 0 KB!")
            os.remove(output_path)
            return False
    return False

# ============================================================
# SIMPLIFIED QA (WITH CONTENT-AWARE NOISE SCORING)
# ============================================================
def comprehensive_quality_check_enhanced(frames_dir, content_type="photo", sample_count=10):
    """
    V13: Content-aware noise scoring
    """
    print_banner("QUALITY ASSURANCE - CONTENT-AWARE (V13.1)")
    frames_list = sorted([f for f in os.listdir(frames_dir) if f.endswith('.png')])
    if len(frames_list) == 0:
        print("No frames!")
        return {}
    
    if len(frames_list) > sample_count:
        indices = np.linspace(0, len(frames_list)-1, sample_count, dtype=int)
        sampled = [frames_list[i] for i in indices]
    else:
        sampled = frames_list
    
    detections = {'ghosting': 0, 'noise': 0}
    noise_variances = []
    noise_scores = []
    prev_frame = None
    
    for idx, frame_name in enumerate(sampled):
        frame = cv2.imread(os.path.join(frames_dir, frame_name))
        if frame is None:
            continue
        
        ghosting_result = detect_ghosting_artifacts(frame, prev_frame)
        if ghosting_result['has_ghosting']:
            detections['ghosting'] += 1
        
        noise_result = detect_noise(frame, content_type)
        if noise_result['has_noise']:
            detections['noise'] += 1
        noise_variances.append(noise_result['variance'])
        noise_scores.append(noise_result['score'])
        
        prev_frame = frame
        print_progress(idx, len(sampled), "QA")
    
    avg_noise_var = np.mean(noise_variances) if noise_variances else 0
    avg_noise_score = np.mean(noise_scores) if noise_scores else 0
    
    print(f"\nRESULTS:")
    for k, v in detections.items():
        pct = v / len(sampled) * 100 if len(sampled) > 0 else 0
        print(f" {k.replace('_', ' ').title()}: {v}/{len(sampled)} ({pct:.1f}%)")
    
    print(f"\nNOISE ANALYSIS:")
    print(f" Average Laplacian Variance: {avg_noise_var:.2f}")
    print(f" Average Noise Score: {avg_noise_score:.2f} (0.0=clean, 1.0=noisy)")
    
    if content_type == "anime":
        print(f" Content Type: Anime (threshold: 1200.0, scoring: 800-1200)")
    elif content_type == "lineart":
        print(f" Content Type: Lineart (threshold: 1500.0, scoring: 1000-1500)")
    else:
        print(f" Content Type: Photo (threshold: 50.0)")
    
    total = sum(detections.values())
    print(f"\nTotal issues: {total}")
    if total == 0:
        print("PERFECT ⭐⭐⭐⭐⭐")
    elif total <= 3:
        print("GOOD ⭐⭐⭐⭐")
    else:
        print("REVIEW ⭐⭐⭐")
    
    print_banner("QA FINISHED", 50)
    return detections

# ============================================================
# MAIN PIPELINE
# ============================================================
if __name__ == "__main__":
    print_banner("WebP to MP4 - V13.1-PERFECTED (FINAL)")
    
    args = get_args("ss.webp", dynamic_mode=True, gamma_profile=1)
    
    durations = extract_frames(args.input_webp)
    
    frame_files = sorted([f for f in os.listdir("frames") if f.endswith(".png")])
    frame_arrays = [cv2.imread(os.path.join("frames", f)) for f in frame_files 
                    if cv2.imread(os.path.join("frames", f)) is not None]
    
    if len(frame_arrays) == 0:
        print("❌ No frames loaded!")
        exit(1)
    
    # DYNAMIC PARAMETER CALCULATION
    if args.dynamic_mode:
        metrics = analyze_frames_for_params(frame_arrays, sample_count=10)
        if metrics:
            dynamic_params = calculate_dynamic_params(metrics, gamma_profile=args.gamma_profile)
            for key, value in dynamic_params.items():
                if hasattr(args, key):
                    setattr(args, key, value)
    
    print("\n" + "="*70)
    print_banner("BEFORE PROCESSING QA (ORIGINAL FRAMES)")
    detections_original = comprehensive_quality_check_enhanced("frames", 
                                                               content_type=args.content_type,
                                                               sample_count=10)
    sample_len = min(10, len(frame_arrays))
    anti_ghosting = detections_original.get('ghosting', 0) / sample_len > 0.3 if sample_len > 0 else False
    print(f"Enabled fixes based on original QA (>30% ghosting):")
    print(f" - Anti-ghosting: {anti_ghosting}")
    
    print("\n" + "="*70)
    compute_metrics("frames", "ORIGINAL")
    print("="*70)
    
    desparkled = frame_arrays
    if args.enable_sparkle:
        desparkled = reduce_jpeg_sparkles_enhanced(
            frame_arrays,
            base_strength=args.sparkle_strength,
            max_strength_cap=args.max_denoise_strength,
            edge_protection_factor=args.edge_protection,
            content_type=args.content_type,
            temporal_window=args.temporal_window
        )
    
    if args.enable_gamma:
        desparkled = fix_gamma_mismatch(desparkled, profile=args.gamma_profile)
        print(f"Gamma fix applied (profile={args.gamma_profile})")
    
    if anti_ghosting:
        desparkled = reduce_ghosting(desparkled)
    
    for i, frame in enumerate(desparkled):
        cv2.imwrite(os.path.join("frames", frame_files[i]), frame)
    
    # Process frames and get masks
    static_mask, semi_static_mask = process_frames_optimized(args)
    
    # V13.1: Apply bilateral polish with improved parameters
    if args.enable_bilateral_polish and static_mask is not None:
        processed_files = sorted([f for f in os.listdir("processed") if f.endswith(".png")])
        processed_frames = [cv2.imread(os.path.join("processed", f)) for f in processed_files]
        
        polished_frames = apply_bilateral_polish(processed_frames, static_mask, 
                                                 semi_static_mask, args.content_type)
        
        # Save polished frames
        for i, frame in enumerate(polished_frames):
            cv2.imwrite(os.path.join("processed", processed_files[i]), frame)
    
    print("\n" + "="*70)
    compute_metrics("processed", "PROCESSED")
    print("="*70)
    
    mp4_success = reassemble_mp4(
        input_dir="processed",
        output_path=args.output_mp4,
        durations=durations,
        crf=args.crf,
        fps_override=args.fps_override
    )
    
    if mp4_success:
        print_banner("AFTER PROCESSING QA (PROCESSED FRAMES)")
        comprehensive_quality_check_enhanced("processed", 
                                            content_type=args.content_type,
                                            sample_count=10)
    
    print("\n" + "="*80)
    print("PIPELINE COMPLETE - V13.1-PERFECTED (FINAL)")
    print("="*80)
    print("\n🎯 V13.1-PERFECTED Features:")
    print("✅ Content-aware noise scoring (accurate QA for anime)")
    print("✅ Adaptive temporal window (5 frames for slow motion)")
    print("✅ Relaxed variance safeguard (-12% for anime)")
    print("✅ Dynamic bilateral sigma (35 for anime, 50 for photo)")
    print("✅ Float32 safety in blending (NaN-proof)")
    print("✅ H.264 codec priority (better compression)")
    print("✅ Production-perfect for anime content")
    print("\n🎊 This is the FINAL version - nothing left to optimize!")



                WebP to MP4 - V13.1-PERFECTED (FINAL)                 

                    CONFIGURATION / ARGUMENT SETUP                    
Input WebP: ss.webp
Output MP4: ultimate_fixed_animation.mp4
Dynamic Mode: True
Bilateral Polish: True
Gamma profile: 1 (fixed)
CRF quality: 18
Verbose mode: True


             STEP 1: EXTRACTING FRAMES FROM ANIMATED WEBP             
Input file path: ss.webp
Creating output directory...
Detected 48 frames in WebP
Beginning extraction loop...
 Extracted 1/48
 Extracted 11/48
 Extracted 21/48
 Extracted 31/48
 Extracted 41/48
 Extracted 48/48

Extraction complete!
Total frames extracted: 48
Median duration per frame: 40.0 ms
Estimated FPS: 25.00

            FRAME EXTRACTION FINISHED             

               ANALYZING FRAMES FOR DYNAMIC PARAMETERS                
Analyzing 10 sample frames...
 Analyzed 1/10
 Analyzed 10/10

📊 Content Metrics:
  Edge Density: 0.1359
  Color Variance: 50.31
  Noise Level: 1106.75
  Temporal Variance: 7.91
  S